# Renyi Differential Privacy

By [Armaan Bhojwani](https://armaanb.net) under [Praneeth Vepakomma](https://praneeth.mit.edu/)

This notebook features some examples of using the dp_accounting Renyi accountant to translate between epsilon, delta and alpha, rho interpretations of Renyi DP, and for usk in compositions.

### Dependencies
- matplotlib
- dp_accounting

### Status
Incomplete

## Setup

In [171]:
import numpy as np

# Privacy parameters
# (https://github.com/google/differential-privacy/blob/main/python/dp_accounting/rdp/rdp_privacy_accountant.py#L781)
custom_orders = False # Set to array of custom orders, otherwise use defaults

sensitivity = 99  # Sensitivity of the function
epsilon = 2    # Privacy garuntee
delta = 10e-7

# Data parameters
data_len = 1500  # Length of dataset
data_low = 0     # Lowest value of dataset
data_high = 99   # Highest value of dataset

# Initialize Numpy RNG
rng = np.random.default_rng()

# Increment data_high so that it includes the value specified
data_high += 1

# Create dataset as defined by above parameters
x = rng.integers(low=data_low, high=data_high, size=(data_len))

## Implementation

In [117]:
import dp_accounting as dpa
import dp_accounting.rdp as rdpa

# noise_multiplier is the ratio of the standard deviation of the Gaussian
# noise to the l2-sensitivity of the function to which it is added
noise_multiplier = dpa.calibrate_dp_mechanism(rdpa.RdpAccountant,
                                              dpa.GaussianDpEvent, epsilon,
                                              delta)

print(f"found noise multiplier: {noise_multiplier}")

found noise multiplier: 2.382578592961857


In [118]:
from common import *

accountant = rdpa.RdpAccountant(
    orders=custom_orders) if custom_orders else rdpa.RdpAccountant()

event = dpa.GaussianDpEvent(noise_multiplier)
accountant = accountant.compose(event)

t_epsilon, alpha = accountant.get_epsilon_and_optimal_order(delta)
t_delta = accountant.get_delta(t_epsilon)

alpha_idx = np.where(accountant._orders == alpha)
rho = accountant._rdp[alpha_idx][0]

print(f"found epsilon: {t_epsilon}")
print(f"found delta:   {t_delta}")
print_hline(40)
print(f"found alpha:   {alpha}")
print(f"found rho:     {rho}")

found epsilon: 1.9999992100966923
found delta:   1.0000000000000023e-06
────────────────────────────────────────
found alpha:   12.0
found rho:     1.0569556863430245


In [170]:
import matplotlib.pyplot as plt

def gaussian_mech_RDP(x, sensitivity, alpha, rho, sigma=0):
    sigma = np.sqrt((sensitivity**2 * alpha) / (2 * rho)) if sigma == 0 else sigma
    print(f"Using sigma: {sigma}")
    return x + np.random.normal(loc=0, scale=sigma)

# https://programming-dp.com/ch6.html#vector-valued-functions-and-their-sensitivities
l2_sensitivity = sensitivity ** 0.5

sum_x = np.sum(x)
sigma = noise_multiplier * l2_sensitivity
sum_X = round(gaussian_mech_RDP(sum_x, l2_sensitivity, alpha, rho, sigma=sigma))

print(f"non-private sum: {sum_x}")
print(f"private sum:     {sum_X}")

Using sigma: 2.382578592961857
non-private sum: 75550
private sum:     75548
